In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
raw_df = pd.read_csv('./data/twitter_user_data.csv', encoding = "ISO-8859-1")
raw_df

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,...,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,...,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,ÛÏIt felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,...,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.587300e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,...,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,...,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.587300e+17,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20045,815757572,True,golden,259,NaN,female,1.0000,yes,1.0,8/5/15 21:16,...,https://pbs.twimg.com/profile_images/656793310...,0,C0DEED,"@lookupondeath ...Fine, and I'll drink tea too...",NaN,783,10/26/15 13:20,6.587400e+17,Verona ªÁ,NaN
20046,815757681,True,golden,248,NaN,male,1.0000,yes,1.0,8/15/12 21:17,...,https://pbs.twimg.com/profile_images/639815429...,0,0,Greg Hardy you a good player and all but don't...,NaN,13523,10/26/15 12:40,6.587300e+17,"Kansas City, MO",NaN
20047,815757830,True,golden,264,NaN,male,1.0000,yes,1.0,9/3/12 1:17,...,https://pbs.twimg.com/profile_images/655473271...,0,C0DEED,You can miss people and still never want to se...,NaN,26419,10/26/15 13:20,6.587400e+17,Lagos Nigeria,NaN
20048,815757921,True,golden,250,NaN,female,0.8489,yes,1.0,11/6/12 23:46,...,https://pbs.twimg.com/profile_images/657716093...,0,0,@bitemyapp i had noticed your tendency to pee ...,NaN,56073,10/26/15 12:40,6.587300e+17,Texas Hill Country,NaN


In [4]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20050 entries, 0 to 20049
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   _unit_id               20050 non-null  int64  
 1   _golden                20050 non-null  bool   
 2   _unit_state            20050 non-null  object 
 3   _trusted_judgments     20050 non-null  int64  
 4   _last_judgment_at      20000 non-null  object 
 5   gender                 19953 non-null  object 
 6   gender:confidence      20024 non-null  float64
 7   profile_yn             20050 non-null  object 
 8   profile_yn:confidence  20050 non-null  float64
 9   created                20050 non-null  object 
 10  description            16306 non-null  object 
 11  fav_number             20050 non-null  int64  
 12  gender_gold            50 non-null     object 
 13  link_color             20050 non-null  object 
 14  name                   20050 non-null  object 
 15  pr

16306/20050 (~81.3%) accounts has a description  

In [36]:
raw_df.drop_duplicates()
# raw_df

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,...,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,...,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,ÛÏIt felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,...,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.587300e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,...,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,...,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.587300e+17,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20045,815757572,True,golden,259,NaN,female,1.0000,yes,1.0,8/5/15 21:16,...,https://pbs.twimg.com/profile_images/656793310...,0,C0DEED,"@lookupondeath ...Fine, and I'll drink tea too...",NaN,783,10/26/15 13:20,6.587400e+17,Verona ªÁ,NaN
20046,815757681,True,golden,248,NaN,male,1.0000,yes,1.0,8/15/12 21:17,...,https://pbs.twimg.com/profile_images/639815429...,0,0,Greg Hardy you a good player and all but don't...,NaN,13523,10/26/15 12:40,6.587300e+17,"Kansas City, MO",NaN
20047,815757830,True,golden,264,NaN,male,1.0000,yes,1.0,9/3/12 1:17,...,https://pbs.twimg.com/profile_images/655473271...,0,C0DEED,You can miss people and still never want to se...,NaN,26419,10/26/15 13:20,6.587400e+17,Lagos Nigeria,NaN
20048,815757921,True,golden,250,NaN,female,0.8489,yes,1.0,11/6/12 23:46,...,https://pbs.twimg.com/profile_images/657716093...,0,0,@bitemyapp i had noticed your tendency to pee ...,NaN,56073,10/26/15 12:40,6.587300e+17,Texas Hill Country,NaN


No duplicated row

In [6]:
raw_df.describe()

,_unit_id,_trusted_judgments,gender:confidence,profile_yn:confidence,fav_number,retweet_count,tweet_count,tweet_id
count,2.005000e+04,20050.000000,20024.000000,20050.000000,20050.000000,20050.000000,2.005000e+04,2.005000e+04
mean,8.157294e+08,3.615711,0.882756,0.993221,4382.201646,0.079401,3.892469e+04,6.587350e+17
std,6.000801e+03,12.331890,0.191403,0.047168,12518.575919,2.649751,1.168371e+05,5.000124e+12
min,8.157192e+08,3.000000,0.000000,0.627200,0.000000,0.000000,1.000000e+00,6.587300e+17
25%,8.157243e+08,3.000000,0.677800,1.000000,11.000000,0.000000,2.398000e+03,6.587300e+17
50%,8.157294e+08,3.000000,1.000000,1.000000,456.000000,0.000000,1.144150e+04,6.587300e+17
75%,8.157345e+08,3.000000,1.000000,1.000000,3315.500000,0.000000,4.002750e+04,6.587400e+17
max,8.157580e+08,274.000000,1.000000,1.000000,341621.000000,330.000000,2.680199e+06,6.587400e+17


In [7]:
raw_df['_golden'].value_counts()

False    20000
True        50
Name: _golden, dtype: int64

Only 50 users were included in the gold standard for the model

In [9]:
raw_df['_unit_state'].value_counts()

finalized    20000
golden          50
Name: _unit_state, dtype: int64

In [11]:
# test = lambda x, y: 1 if not x and y=='finalized' or x and y=='golden' else 0

In [13]:
# raw_df['golden_to_unit_state'] = raw_df.apply(lambda x: test(x._golden, x._unit_state), axis=1)

In [14]:
# raw_df['golden_to_unit_state'].value_counts()

1    20050
Name: golden_to_unit_state, dtype: int64

In [15]:
# raw_df = raw_df.drop(['golden_to_unit_state'], axis=1)

Column `_golden` and column `_unit_state` 100% proportionally correlated

In [16]:
raw_df['_trusted_judgments'].value_counts()

3      20000
259        4
245        4
249        4
264        4
260        3
242        3
227        2
235        2
247        2
262        1
273        1
248        1
240        1
212        1
252        1
253        1
271        1
237        1
270        1
239        1
246        1
256        1
274        1
231        1
254        1
261        1
255        1
233        1
228        1
258        1
250        1
Name: _trusted_judgments, dtype: int64

20000 non-golden users get 3, other 50 get different numbers

In [21]:
raw_df['_last_judgment_at'].value_counts()

10/26/15 23:05    217
10/26/15 23:07    205
10/26/15 23:31    182
10/26/15 23:57    181
10/27/15 1:43     169
                 ... 
10/27/15 2:29       2
10/27/15 2:21       1
10/27/15 2:48       1
10/27/15 2:17       1
10/27/15 2:18       1
Name: _last_judgment_at, Length: 283, dtype: int64

In [23]:
raw_df['gender'].value_counts()

female     6700
male       6194
brand      5942
unknown    1117
Name: gender, dtype: int64

In [24]:
raw_df['gender'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 20050 entries, 0 to 20049
Series name: gender
Non-Null Count  Dtype 
--------------  ----- 
19953 non-null  object
dtypes: object(1)
memory usage: 156.8+ KB


5 categories for column `gender`: male, female, brand, unknown, NULL

In [28]:
raw_df['gender:confidence'].value_counts().to_dict()

{1.0: 13926,
 0.0: 71,
 0.6691: 31,
 0.6709: 30,
 0.6591: 30,
 0.6667: 30,
 0.6619: 29,
 0.6743: 27,
 0.6686: 27,
 0.6769: 26,
 0.6666: 25,
 0.6678: 24,
 0.6561: 24,
 0.6642: 24,
 0.6715: 24,
 0.6707: 24,
 0.6656: 24,
 0.6752: 24,
 0.6609: 23,
 0.6675: 23,
 0.6685: 22,
 0.6774: 22,
 0.6711: 22,
 0.6645: 22,
 0.6766: 22,
 0.6741: 22,
 0.6674: 21,
 0.6739: 21,
 0.6706: 21,
 0.6659: 21,
 0.6617: 21,
 0.6664: 21,
 0.6796: 20,
 0.6653: 20,
 0.6705: 20,
 0.6587: 20,
 0.6787: 20,
 0.6641: 20,
 0.6866: 20,
 0.6628: 20,
 0.6702: 20,
 0.6742: 20,
 0.6708: 20,
 0.6688: 20,
 0.6611: 19,
 0.6692: 19,
 0.667: 19,
 0.6848: 19,
 0.6698: 19,
 0.6626: 19,
 0.6745: 19,
 0.6634: 19,
 0.6736: 19,
 0.6727: 18,
 0.675: 18,
 0.6801: 18,
 0.6672: 18,
 0.6509: 18,
 0.6655: 18,
 0.6602: 18,
 0.6722: 18,
 0.6723: 18,
 0.6648: 18,
 0.6755: 18,
 0.6734: 17,
 0.6735: 17,
 0.6757: 17,
 0.6684: 17,
 0.6772: 17,
 0.6636: 17,
 0.6829: 17,
 0.6713: 17,
 0.6763: 17,
 0.6638: 17,
 0.6623: 16,
 0.6633: 16,
 0.6818: 16,
 0.6

In [29]:
from collections import OrderedDict

In [30]:
OrderedDict(sorted(raw_df['gender:confidence'].value_counts().to_dict().items()))

OrderedDict([(0.0, 71),
             (0.314, 1),
             (0.3206, 1),
             (0.3223, 2),
             (0.3268, 1),
             (0.3269, 1),
             (0.3272, 1),
             (0.3275, 2),
             (0.3276, 1),
             (0.3282, 1),
             (0.3283, 1),
             (0.3286, 1),
             (0.3303, 1),
             (0.3307, 1),
             (0.3311, 1),
             (0.3312, 1),
             (0.3319, 1),
             (0.332, 1),
             (0.3321, 1),
             (0.3322, 1),
             (0.3327, 1),
             (0.333, 1),
             (0.3333, 1),
             (0.3334, 1),
             (0.3335, 1),
             (0.3337, 1),
             (0.3339, 1),
             (0.334, 3),
             (0.3343, 1),
             (0.3347, 1),
             (0.3348, 2),
             (0.3349, 1),
             (0.335, 5),
             (0.3351, 2),
             (0.3352, 2),
             (0.3353, 3),
             (0.3354, 5),
             (0.3355, 2),
             (0.335

13926/20050 (~69.5%) guesses with 100% confidence

In [32]:
raw_df['profile_yn'].value_counts()

yes    19953
no        97
Name: profile_yn, dtype: int64

97 accounts not available at judging time

In [33]:
OrderedDict(sorted(raw_df['profile_yn:confidence'].value_counts().to_dict().items()))

OrderedDict([(0.6272, 1),
             (0.6308, 1),
             (0.631, 1),
             (0.6323, 1),
             (0.6361, 1),
             (0.6362, 1),
             (0.6373, 1),
             (0.6377, 1),
             (0.6403, 1),
             (0.6404, 1),
             (0.6406, 1),
             (0.6408, 1),
             (0.641, 1),
             (0.6418, 1),
             (0.6424, 1),
             (0.6426, 1),
             (0.643, 1),
             (0.6432, 1),
             (0.6434, 1),
             (0.6436, 4),
             (0.644, 2),
             (0.6442, 1),
             (0.6444, 1),
             (0.6445, 1),
             (0.6452, 1),
             (0.6456, 1),
             (0.646, 1),
             (0.6461, 1),
             (0.6462, 1),
             (0.6464, 1),
             (0.6466, 1),
             (0.6467, 1),
             (0.647, 1),
             (0.6471, 1),
             (0.6475, 1),
             (0.6476, 1),
             (0.6477, 1),
             (0.6478, 2),
             (0.64

No account was guess with the confidence less than 60%  
19642/20050 (~98%) guesses with 100% confidence

In [34]:
19642/20050

0.9796508728179552

In [37]:
raw_df['gender_gold'].value_counts()

male               19
female             14
brand              13
unknown             2
female\nunknown     1
male\nunknown       1
Name: gender_gold, dtype: int64

In [38]:
raw_df['profile_yn_gold'].value_counts()

yes    50
Name: profile_yn_gold, dtype: int64

100% golden accounts were available at judging time

In [39]:
raw_df['tweet_coord'].value_counts()

[37.7463, -25.6668]            2
[52.2297, 21.0122]             2
[30.764, 20.2236]              2
[25.2803, 51.5225]             2
[53.33422016, -2.74855379]     2
                              ..
[45.53106791, -122.6503049]    1
[33.4931, -111.926]            1
[43.0621, 141.354]             1
[42.93237349, -70.88862593]    1
[32.725409, -97.3208496]       1
Name: tweet_coord, Length: 146, dtype: int64

In [41]:
raw_df['tweet_coord'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 20050 entries, 0 to 20049
Series name: tweet_coord
Non-Null Count  Dtype 
--------------  ----- 
159 non-null    object
dtypes: object(1)
memory usage: 156.8+ KB


159 accounts have location turned on

In [42]:
raw_df['tweet_count'].value_counts()

151386     30
1          26
3          24
2372591    24
2          21
           ..
20977       1
27483       1
12728       1
8563        1
17851       1
Name: tweet_count, Length: 14280, dtype: int64

In [43]:
raw_df['user_timezone'].value_counts()

Eastern Time (US & Canada)        2496
Pacific Time (US & Canada)        2106
Central Time (US & Canada)        1505
London                            1307
Atlantic Time (Canada)             589
                                  ... 
America/Boise                        1
Ulaan Bataar                         1
Europe/Moscow                        1
GMT+3                                1
America/Argentina/Buenos_Aires       1
Name: user_timezone, Length: 156, dtype: int64

In [44]:
raw_df['user_timezone'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 20050 entries, 0 to 20049
Series name: user_timezone
Non-Null Count  Dtype 
--------------  ----- 
12252 non-null  object
dtypes: object(1)
memory usage: 156.8+ KB


12252/20050 (~61%) accounts has timezone information

In [46]:
12252/20050

0.611072319201995